In [2]:
import pandas as pd
import numpy as np
import os

### Carga y filtrado de los datos en distintos niveles

In [3]:
df = pd.read_parquet('../api/data/solar.parquet')
df.head()

,Customer,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,3:00,3:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,CL,1/07/2012,1.250,1.250,1.250,1.263,0.131,0.569,0.556,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.081
1,1,GC,1/07/2012,0.855,0.786,0.604,0.544,0.597,0.612,1.245,...,0.329,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.090,0.068
2,1,GG,1/07/2012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,CL,2/07/2012,1.250,1.250,1.125,0.000,0.925,0.231,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.069
4,1,GC,2/07/2012,0.309,0.082,0.059,0.097,0.290,0.155,0.703,...,0.696,0.353,0.464,0.229,0.811,0.222,0.306,1.034,0.136,0.067


In [64]:
dfmod = df.melt(id_vars=['Customer', 'Consumption Category', 'date'], value_name='amount', var_name='hour')

df2 = dfmod.copy()
df2['Datetime'] = pd.to_datetime(df2['date'] + df2['hour'], format='%d/%m/%Y%H:%M')
df2.drop(columns=['date', 'hour'], inplace=True)

df2.head()
# gc_serie = df2[df2['Consumption Category'] == 'GC']['amount']
# cl_serie = df2[df2['Consumption Category'] == 'CL']['amount']
# df2.sort_values('Datetime', inplace=True)
# df2.set_index('Datetime', inplace=True)

,Customer,Consumption Category,amount,Datetime
0,1,CL,1.250,2012-07-01 00:30:00
1,1,GC,0.855,2012-07-01 00:30:00
2,1,GG,0.000,2012-07-01 00:30:00
3,1,CL,1.250,2012-07-02 00:30:00
4,1,GC,0.309,2012-07-02 00:30:00


In [54]:
df3 = df2.pivot_table(values='amount', columns='Consumption Category', index=['Customer', 'Datetime'])
df3.rename_axis(columns=None, inplace=True)
# df3.head()

CL     GC   GG
Customer Datetime                              
1        2012-07-01 00:00:00  1.081  0.068  0.0
         2012-07-01 00:30:00  1.250  0.855  0.0
         2012-07-01 01:00:00  1.250  0.786  0.0
         2012-07-01 01:30:00  1.250  0.604  0.0
         2012-07-01 02:00:00  1.263  0.544  0.0

Existen usuarios sin datos en las columnas de CL
Se debe reemplazar por 0 debido a que no cuentan con consumos

In [51]:
print(df3['CL'].isna().sum())
print(df3['GC'].isna().sum())
print(df3['GG'].isna().sum())

2865600
0
0


In [63]:
df4 = df3.fillna(value=0)
print(df4['CL'].isna().sum())
print(df4['GC'].isna().sum())
print(df4['GG'].isna().sum())
df4.reset_index(inplace=True)
df4.dtypes


0
0
0


Customer             int64
Datetime    datetime64[ns]
CL                 float64
GC                 float64
GG                 float64
dtype: object

### Guardar los DataFrames en archivos parquet
    Se busca comparar el espacio que consume cada DataFrame

In [ ]:
files = ['melt.parquet', 'melt_datetime.parquet', 'pivot.parquet']

dfmod.to_parquet('melt.parquet')

df2.to_parquet('melt_datetime.parquet')

df4.to_parquet('pivot.parquet')


In [62]:
for name in files:
    print(f'{name:<24}{os.stat(name).st_size}')

melt.parquet            25218502
melt_datetime.parquet   26145940
pivot.parquet           22625916


In [65]:
pd.read_parquet('pivot.parquet')

,Customer,Datetime,CL,GC,GG
0,1,2012-07-01 00:00:00,1.081,0.068,0.0
1,1,2012-07-01 00:30:00,1.250,0.855,0.0
2,1,2012-07-01 01:00:00,1.250,0.786,0.0
3,1,2012-07-01 01:30:00,1.250,0.604,0.0
4,1,2012-07-01 02:00:00,1.263,0.544,0.0
...,...,...,...,...,...
5252107,300,2013-06-30 21:30:00,0.000,0.646,0.0
5252108,300,2013-06-30 22:00:00,0.000,0.581,0.0
5252109,300,2013-06-30 22:30:00,0.000,0.458,0.0
5252110,300,2013-06-30 23:00:00,0.000,0.472,0.0
